# Setup
Initializes connections to mongodb.

In [1]:
%load_ext autoreload
%autoreload 2

from bs4 import BeautifulSoup
from pymongo import MongoClient
import pprint
from parser import *
import os
from selenium import webdriver
from time import sleep
import re

client = MongoClient("localhost", 27017)
db = client.retraction_watch
retracted = db.retracted

In [29]:
d = Counter()

# Scrape RetractionWatchDB
Executes scraper to insert into mongodb

In [36]:
d["S"]

2

In [35]:
d["S"] += 1

In [2]:
driver = webdriver.Chrome("chromedriver")
driver.get("http://retractiondatabase.org")

Select Reason(s) for retraction
 - concerns about data
 - concerns about results
 - conflict of interest
 - doint the right thing
 - euphamisms for misconduct
 - falsification/fabrication of data
 - falsification/fabrication of results
 - misconduct by author

In [29]:
html = driver.page_source
soup = BeautifulSoup(html, "lxml")
read_to_mongo(soup, retracted)

# Get full text of retracted articles

In [39]:
retracted_pubmed = list(retracted.find(
    {"publication.pubmed_id":{"$exists":1,"$ne":"00000000"},
     "links": {"$exists":0},
     "text": {"$exists":0}}))

In [40]:
len(retracted_pubmed)

54

In [4]:
retracted_pubmed[0].keys()

dict_keys(['_id', 'rwdb_id', 'title', 'subjects', 'journal', 'publisher', 'institutions', 'reasons', 'authors', 'publication', 'retraction', 'doc_types', 'notice_type', 'countries', 'notes'])

In [13]:
pubmed_ids = [ a["publication"]["pubmed_id"] for a in retracted_pubmed ]


## Sanity check

In [14]:
print(len(pubmed_ids), pubmed_ids[0])
print(len(set(pubmed_ids)))


54 7862147
53


In [240]:
s = set()
duplicates = []
for n in pubmed_ids:
    if n not in s:
        s.add(n)
    else:
        duplicates.append(n)
sorted(duplicates)[0]

'10050223'

In [241]:
wtf = retracted.find({"publication.pubmed_id":"10050223"}, {"_id":0})

In [242]:
def compare(docs, attr):
    first = docs[0][attr]
    same = [first==doc[attr] for doc in docs]
    return all(same)
    
wtf = list(wtf)
print("Same title:",compare(wtf,"title"))
print("Same journal:",compare(wtf,"journal"))
print("Same notice type:", compare(wtf, "notice_type"))

Same title: True
Same journal: True
Same notice type: False


In [230]:
print("Notice types:",wtf[0]["notice_type"],",",wtf[1]["notice_type"])

Notice types: Retraction , Expression of concern


Most are like that. Multiple notices on the same article. Be advised to avoid duplicating topics.

Some are corrupted IDs, missing a digit and lead nowhere. These should be caught by the parser.

## Actually insert

In [10]:
driver = webdriver.Chrome("chromedriver")
for pm_id in set(pubmed_ids):
    fulltext = get_fulltext(pm_id, driver)
    if not fulltext:
        continue
    retracted.update_many(
        {"publication.pubmed_id":pm_id},
        {"$set": fulltext}
    )

In [46]:
pubmed_ids[0]

'7862147'

In [44]:
list(retracted.find({"publication.pubmed_id":pubmed_ids[0]}))

[{'_id': ObjectId('5b099c278e32411cbf535316'),
  'authors': ['Maria Teresa Berlingieri',
   'Guidalberto Manfioletti',
   'Massimo Santoro',
   'Antonella Bandiera',
   'Roberta Visconti',
   'Vincenzo Giancotti',
   'Alfredo Fusco'],
  'countries': ['Italy'],
  'doc_types': ['Research Article'],
  'institutions': ['Centro di Endocrinologia ed Oncologia Sperimentale, Consiglio Nazionale delle Ricerche, Facoltà di Medicina e Chirurgia, Università degli Studi di Napoli, Italy'],
  'journal': 'Molecular and Cellular Biology ---American Society',
  'notes': 'claire   176 cites   http://mcb.asm.org/content/38/6/e00662-17.full?sid=2e59bfe3-3ba3-42c8-9bad-5ad00ec2f9d9\n\nVolume 15, no. 3, p. 1545–1553, 1995, https://doi.org/10.1128/MCB.15.3.1545. The publisher hereby retracts this article. Questions have been raised by concerned readers about the integrity of the data. The American Society for Microbiology has reviewed the figures and confirmed evidence of apparent manipulation and duplicatio

## Which parser should I build next?

In [252]:
pubmed_ids[0]

'27687725'

In [21]:
from collections import Counter

def trim_url(url):
    url = re.sub(r"^https?://", "", url)
    url = re.sub(r"/.*","",url)
    return url

a = "https://github.com/arstarsin/arosden"
print(trim_url(a))

stems = Counter()

In [46]:
for i in range(100):
    pubmed_id = retracted_pubmed[i]["publication"]["pubmed_id"]
    sleep(5)
    links = fulltext_links(pubmed_id)
    print(i)
    links = [trim_url(link) for link in links]
    if "www.ncbi.nlm.nih.gov" in links:
        stems["www.ncbi.nlm.nih.gov"] += 1
    else:
        for link in links:
            stems[link] += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [47]:
print(stems)

Counter({'www.ncbi.nlm.nih.gov': 45, 'linkinghub.elsevier.com': 13, 'dx.doi.org': 10, 'Insights.ovid.com': 6, 'jamanetwork.com': 4, 'jcs.biologists.org': 4, 'www.neurology.org': 2, 'www.karger.com?DOI=10.1159': 2, 'diabetes.diabetesjournals.org': 2, 'www.researchprotocols.org': 1, 'archpedi.jamanetwork.com': 1, 'archinte.jamanetwork.com': 1, 'www.reproduction-online.org': 1, 'circheartfailure.ahajournals.org': 1, 'www.tandfonline.com': 1, 'joi.jlc.jst.go.jp': 1, 'journals.sagepub.com': 1, 'www.jbc.org': 1, 'www.liebertonline.com': 1})


In [68]:
pm_ids = [p["publication"]["pubmed_id"] for p in retracted_pubmed]
fulltexts = [get_fulltext(a) for a in pm_ids[:2]]


In [86]:
driver = webdriver.Chrome("chromedriver")

In [91]:
pm_ids = [p["publication"]["pubmed_id"] for p in retracted_pubmed]
fulltexts = [se_fulltext(a) for a in pm_ids[:10]]
